In [1]:

#For stacking 
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19880 sha256=908211130c926f9881643f9db2e80e59b55b7767336b00197705bdc8b4e4434f
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [2]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from collections import Counter #for Smote, 
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2,f_classif
from sklearn.svm import SVR


import warnings
warnings.filterwarnings("ignore")


In [3]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive




Mounted at /gdrive
/gdrive


In [4]:
trainfile = r'/gdrive/Shared drives/The Big Four/CIS 508/House Pricing/train.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/Shared drives/The Big Four/CIS 508/House Pricing/test.csv'
test_data = pd.read_csv(testfile)

In [5]:
print(train_data.shape)
print(test_data.shape)

(1460, 81)
(1459, 80)


In [6]:
# Finding Missing Values in each variable

# checking missing values in each columns
def missing_values(df):
    miss_val=df.isnull().sum()
    miss_val_perc=100*df.isnull().sum()/len(df)
    miss_val_table=pd.concat([miss_val,miss_val_perc],axis=1)
    miss_val_table_rename=miss_val_table.rename(columns={0:"Missing Values",1:"% Total Values"})
    miss_val_table_rename=miss_val_table_rename[miss_val_table_rename.iloc[:,1]!=0].sort_values('% Total Values',ascending=False).round(2)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(miss_val_table_rename.shape[0]) +
             " columns that have missing values.")
    return miss_val_table_rename

In [7]:
missing_values_table=missing_values(train_data)
missing_values_table.head(30)

Your selected dataframe has 81 columns.
There are 19 columns that have missing values.


,Missing Values,% Total Values
PoolQC,1453,99.52
MiscFeature,1406,96.30
Alley,1369,93.77
Fence,1179,80.75
FireplaceQu,690,47.26
LotFrontage,259,17.74
GarageType,81,5.55
GarageYrBlt,81,5.55
GarageFinish,81,5.55
GarageQual,81,5.55


In [8]:
#Copy Train data excluding target
X0 = train_data.iloc[:, :-1].copy()
y=train_data["SalePrice"]
y = train_data.iloc[:, -1]

In [9]:
#Combine Train data and test data first so that the SAME COLUMNS are DROPPED in each
combined_Data = pd.concat([X0, test_data], keys=[0,1])

combined_Data.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      486
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64

In [10]:
## Check the outliers and the index number

#combined_Data['GrLivArea'].sort_values(ascending=False).head(2)
#combined_Data['TotalBsmtSF'].sort_values(ascending=False).head(1)
#combined_Data['MasVnrArea'].sort_values(ascending=False).head(1)
#combined_Data['1stFlrSF'].sort_values(ascending=False).head(1)



In [11]:

# Removing the outliers by the index numbers in the Train and test both
pos = [1298,523, 297]
combined_Data.drop(combined_Data.index[pos], inplace=True)

pos = [1298,523, 297]
y.drop(y.index[pos], inplace=True)

In [12]:
# Check if the number of rows deducted are 3 in both the datasets

print(y.shape)
print(combined_Data.shape)

(1457,)
(2916, 80)


In [13]:
#Define threshold for dropping columns
percent=int(0*(combined_Data.shape[0]))
print(percent)
#Drop columns that have less than "thresh" number of non_Nans
td1=combined_Data.dropna(thresh=percent,axis=1)
print(td1.shape)

0
(2916, 80)


In [14]:
#NOW IMPUTE MISSING VALUES FOR THE OTHER COLUMNS=========================
#IMPUTE (SUBSTITUTE) MEAN VALUES FOR NaN IN NUMERIC COLUMNS 
numeric=td1.select_dtypes(include=['int','float64']).columns
for num in numeric:
  td1[num]=td1[num].fillna(td1[num].mean())

#IMPUTE (SUBSTITUTE) MODE VALUES FOR NaN IN CATEGORICAL COLUMNS
train_cat_cols = td1.select_dtypes(exclude=['int','float64']).columns#selecting the categorical columns
for colss in train_cat_cols:
  if(td1.iloc[0][colss]=="N"):
        td1[colss]=td1[colss].fillna("N")
  else:
    td1[colss]=td1[colss].fillna(td1[colss].mode())
  
print(td1.head(20))

      Id  MSSubClass MSZoning  ...  YrSold  SaleType SaleCondition
0 0    1          60       RL  ...    2008        WD        Normal
  1    2          20       RL  ...    2007        WD        Normal
  2    3          60       RL  ...    2008        WD        Normal
  3    4          70       RL  ...    2006        WD       Abnorml
  4    5          60       RL  ...    2008        WD        Normal
  5    6          50       RL  ...    2009        WD        Normal
  6    7          20       RL  ...    2007        WD        Normal
  7    8          60       RL  ...    2009        WD        Normal
  8    9          50       RM  ...    2008        WD       Abnorml
  9   10         190       RL  ...    2008        WD        Normal
  10  11          20       RL  ...    2008        WD        Normal
  11  12          60       RL  ...    2006       New       Partial
  12  13          20       RL  ...    2008        WD        Normal
  13  14          20       RL  ...    2007       New       Par

In [15]:
#CHECK IF THERE ARE ANY REMAINING MISSING VALUES 
td1.isnull().sum()
#print(td1.shape)

Id               0
MSSubClass       0
MSZoning         4
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         1
SaleCondition    0
Length: 80, dtype: int64

In [16]:
# td1['BsmtCond'].fillna('None', inplace=True)
# td1['BsmtQual'].fillna('None', inplace=True)
# td1['BsmtExposure'].fillna('None', inplace=True)
# td1['BsmtFinType1'].fillna('None', inplace=True)
# td1['BsmtFinType2'].fillna('None', inplace=True)
# td1['Electrical'].fillna('oth', inplace=True)
# td1['GarageType'].fillna('None', inplace=True)
# td1['GarageFinish'].fillna('None', inplace=True)
# td1['GarageQual'].fillna('None', inplace=True)
# td1['GarageCond'].fillna('None', inplace=True)
# td1['MasVnrType'].fillna('None', inplace=True)
# td1['MSZoning'].fillna('None', inplace=True)
# td1['Utilities'].fillna('None', inplace=True)
# td1['Exterior1st'].fillna('None', inplace=True)
# td1['Exterior2nd'].fillna('None', inplace=True)
# td1['Functional'].fillna('Typ', inplace=True)
# td1['KitchenQual'].fillna('Typ', inplace=True)
# td1['SaleType'].fillna('Oth', inplace=True)
# td1['PoolQC'].fillna('None', inplace=True)
# td1['MiscFeature'].fillna('None', inplace=True)
# td1['Alley'].fillna('None', inplace=True)
# td1['Fence'].fillna('None', inplace=True)
# td1['FireplaceQu'].fillna('None', inplace=True)

In [17]:
missing_values_table=missing_values(td1)
missing_values_table.head(30)

Your selected dataframe has 80 columns.
There are 23 columns that have missing values.


,Missing Values,% Total Values
PoolQC,2907,99.69
MiscFeature,2811,96.40
Alley,2719,93.24
Fence,2345,80.42
FireplaceQu,1420,48.70
GarageQual,159,5.45
GarageFinish,159,5.45
GarageCond,159,5.45
GarageType,157,5.38
BsmtCond,82,2.81


In [18]:
print(td1.shape)

(2916, 80)


In [19]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
#y["SalePrice"] = np.log1p(y["SalePrice"])

#td1.drop('YearBuilt', axis=1, inplace=True)

In [20]:
#Separate Train data and test data
X = td1.xs(0)
testData = td1.xs(1)
print(X.shape)
print(y.shape)
print(testData.shape)

(1457, 80)
(1457,)
(1459, 80)


In [21]:
#List of Categorical Features
categoricalFeatures = ["PoolQC","MiscFeature","Alley","Fence","FireplaceQu","MSSubClass","MSZoning", "Street","LotShape","LandContour","Utilities","LotConfig","LandSlope","Neighborhood","Condition1","Condition2","BldgType","HouseStyle","OverallQual","OverallCond","RoofStyle","RoofMatl","Exterior1st","Exterior2nd","MasVnrType","ExterQual","ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","Heating","HeatingQC","CentralAir","Electrical","KitchenQual","Functional","GarageType","GarageFinish","GarageQual","GarageCond","PavedDrive","SaleType","SaleCondition"]


#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([X,testData], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)

#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)

X_test.head()

,Id,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,...,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,80.0,11622,1961,1961,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1961.0,1.0,730.0,140,0,0,0,120,0,0,6,2010,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,1462,81.0,14267,1958,1958,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1958.0,1.0,312.0,393,36,0,0,0,0,12500,6,2010,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,1463,74.0,13830,1997,1998,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,1997.0,2.0,482.0,212,34,0,0,0,0,0,3,2010,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,1464,78.0,9978,1998,1998,20.0,602.0,0.0,324.0,926.0,926,678,0,1604,0.0,0.0,2,1,3,1,7,1,1998.0,2.0,470.0,360,36,0,0,0,0,0,6,2010,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,1465,43.0,5005,1992,1992,0.0,263.0,0.0,1017.0,1280.0,1280,0,0,1280,0.0,0.0,2,0,2,1,5,0,1992.0,2.0,506.0,0,82,0,0,144,0,0,1,2010,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [22]:
pd.concat([X_train],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/House Pricing/X_out_3.csv", index = None)
res=pd.read_csv("/gdrive/Shared drives/The Big Four/CIS 508/House Pricing/X_out_3.csv")
res.head()

pd.concat([y],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/House Pricing/y_out_3.csv", index = None)
res=pd.read_csv("/gdrive/Shared drives/The Big Four/CIS 508/House Pricing/y_out_3.csv")
res.head()


pd.concat([X_test],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/House Pricing/testData_out_3.csv", index = None)
res=pd.read_csv("/gdrive/Shared drives/The Big Four/CIS 508/House Pricing/testData_out_3.csv")
res.head()

,Id,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,...,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,80.0,11622,1961,1961,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1961.0,1.0,730.0,140,0,0,0,120,0,0,6,2010,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,1462,81.0,14267,1958,1958,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1958.0,1.0,312.0,393,36,0,0,0,0,12500,6,2010,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,1463,74.0,13830,1997,1998,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,1997.0,2.0,482.0,212,34,0,0,0,0,0,3,2010,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,1464,78.0,9978,1998,1998,20.0,602.0,0.0,324.0,926.0,926,678,0,1604,0.0,0.0,2,1,3,1,7,1,1998.0,2.0,470.0,360,36,0,0,0,0,0,6,2010,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,1465,43.0,5005,1992,1992,0.0,263.0,0.0,1017.0,1280.0,1280,0,0,1280,0.0,0.0,2,0,2,1,5,0,1992.0,2.0,506.0,0,82,0,0,144,0,0,1,2010,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [23]:

#-------------------------------RUN FROM HERE--------------------------------#

csv1 = r'/gdrive/Shared drives/The Big Four/CIS 508/House Pricing/X_out_3.csv'
X_train = pd.read_csv(csv1)

csv2 = r'/gdrive/Shared drives/The Big Four/CIS 508/House Pricing/y_out_3.csv'
y = pd.read_csv(csv2)

csv3 = r'/gdrive/Shared drives/The Big Four/CIS 508/House Pricing/testData_out_3.csv'
X_test = pd.read_csv(csv3)

In [24]:
#Decision Tree Regressor ========================================================================
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeRegressor()
clf.fit(X_train, y)

clf_predict_Test=clf.predict(X_test)

#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(clf.predict(X_test),columns=["SalePrice"])

pred.head()
#Write into a file with actual prediction and corresponding probability
pd.concat([X_test['Id'],pred],axis=1).to_csv(r'/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles/DefaultDecisionTree_Results.csv', index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles/DefaultDecisionTree_Results.csv')
res.head()
# #Save predictions
# df_DT=pd.DataFrame()
# df_DT['Sale']=clf_predict_Test
# export_csv = df_DT.to_csv(r'/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles/DefaultDecisionTree_Results.csv')


,Id,SalePrice
0,1461,104900.0
1,1462,84500.0
2,1463,196500.0
3,1464,163900.0
4,1465,192000.0


In [25]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeRegressor(**grid_parm)
clf.fit(X_train,y)

pred=pd.DataFrame(clf.predict(X_test),columns=["SalePrice"])

pd.concat([X_test['Id'],pred],axis=1).to_csv(r'/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles/HyperDecisionTree_Results.csv', index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles/HyperDecisionTree_Results.csv')
res.head()

{'min_samples_split': 20, 'max_depth': 13}


,Id,SalePrice
0,1461,109430.000000
1,1462,84500.000000
2,1463,192638.461538
3,1464,177800.000000
4,1465,213377.250000


In [26]:
#Random Forest Regressor==============================================================================

rfc = RandomForestRegressor()
rfc.fit(X_train, y)

rfc_predict_Test=rfc.predict(X_test)

#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(clf.predict(X_test),columns=["SalePrice"])

pred.head()
#Write into a file with actual prediction and corresponding probability
pd.concat([X_test['Id'],pred],axis=1).to_csv(r'/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles/DefaultRandomForest_Results.csv', index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles/DefaultRandomForest_Results.csv')
res.head()

,Id,SalePrice
0,1461,109430.000000
1,1462,84500.000000
2,1463,192638.461538
3,1464,177800.000000
4,1465,213377.250000


In [29]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestRegressor(**grid_parm_rfc)
rfc.fit(X_train,y)

pred=pd.DataFrame(clf.predict(X_test),columns=["SalePrice"])

pred.head()
#Write into a file with actual prediction and corresponding probability
pd.concat([X_test['Id'],pred],axis=1).to_csv(r'/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/HyperRandomForest_Results.csv', index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/HyperRandomForest_Results.csv')
res.head()

{'n_estimators': 130, 'min_samples_split': 10, 'max_depth': 15}


,Id,SalePrice
0,1461,109430.000000
1,1462,84500.000000
2,1463,192638.461538
3,1464,177800.000000
4,1465,213377.250000


In [30]:
#Gradient Boosting Regressor================================================================================

abc =GradientBoostingRegressor()
abc.fit(X_train, y)
pred=pd.DataFrame(abc.predict(X_test),columns=["SalePrice"])

pd.concat([X_test["Id"],pred],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/DefaultGB_Results.csv", index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/DefaultGB_Results.csv')
print(res.head)

<bound method NDFrame.head of         Id      SalePrice
0     1461  121997.803968
1     1462  163575.300198
2     1463  183287.304466
3     1464  183764.168856
4     1465  180882.102621
...    ...            ...
1454  2915   85350.385876
1455  2916   88376.707438
1456  2917  146255.037603
1457  2918  133081.225464
1458  2919  220881.566883

[1459 rows x 2 columns]>


In [31]:
search_grid={'n_estimators':range(500,1000,10),'learning_rate':[0.001,0.005]}
#Random Search for hyperparameter tuning
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, y)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)

#Construct Gradient Boosting Trees using the best parameters
abc= GradientBoostingRegressor(**grid_parm_abc)
abc.fit(X_train, y)

#Get the probability of each X_test value is succeed
pred =pd.DataFrame(abc.predict(X_test),columns=["SalePrice"])
#print the probability of the target is success
print(pred.head())

pd.concat([X_test["Id"],pred],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/HyperGB_Results.csv", index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/HyperGB_Results.csv')
print(res.head)

{'n_estimators': 990, 'learning_rate': 0.005}
       SalePrice
0  123862.099199
1  161969.814476
2  188699.370692
3  180450.988862
4  184461.019785
<bound method NDFrame.head of         Id      SalePrice
0     1461  123862.099199
1     1462  161969.814476
2     1463  188699.370692
3     1464  180450.988862
4     1465  184461.019785
...    ...            ...
1454  2915   89699.170439
1455  2916   99370.074770
1456  2917  160378.492797
1457  2918  135470.110679
1458  2919  225062.621365

[1459 rows x 2 columns]>


In [32]:
#MLP classifier
#Default mode
mlp = MLPRegressor()
mlp.fit(X_train,y)

#Get the probability of each X_test value is succeed
pred =pd.DataFrame(mlp.predict(X_test),columns=['SalePrice'])


pd.concat([X_test['Id'],pred],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/DefaultMLP_Results.csv", index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/DefaultMLP_Results.csv')
res.head()

,Id,SalePrice
0,1461,141174.142887
1,1462,160897.062629
2,1463,193474.407795
3,1464,192840.388060
4,1465,168588.098453


In [33]:
#Hyperparameter tuning for MLP
parameters={ 'hidden_layer_sizes': range(80,150,20),'max_iter' : range(10,200,50)}
mlp_random = RandomizedSearchCV(mlp,parameters,n_iter=15)
mlp_random.fit(X_train, y)
grid_parm_mlp=mlp_random.best_params_
print(grid_parm_mlp)

#contruct random forest using the best parameters
mlp= MLPRegressor(**grid_parm_mlp)
mlp.fit(X_train,y)

#Get the probability of each X_test value is succeed
pred =pd.DataFrame(mlp.predict(X_test),columns=["SalePrice"])
#print the probability of the target is success
print(pred.head())

pd.concat([X_test["Id"],pred],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/HyperMLP_Results.csv", index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/HyperMLP_Results.csv')
print(res.head)

{'max_iter': 160, 'hidden_layer_sizes': 140}
       SalePrice
0  134779.303326
1  162601.133714
2  190624.635934
3  189762.337274
4  164168.996147
<bound method NDFrame.head of         Id      SalePrice
0     1461  134779.303326
1     1462  162601.133714
2     1463  190624.635934
3     1464  189762.337274
4     1465  164168.996147
...    ...            ...
1454  2915   78569.053960
1455  2916   95262.610752
1456  2917  191601.903342
1457  2918   95506.809431
1458  2919  219810.453664

[1459 rows x 2 columns]>


In [34]:
#Support Vector Regression
svr = SVR()
svr.fit(X_train, y)

#Get the probability of each X_test value is succeed
pred =pd.DataFrame(svr.predict(X_test),columns=['SalePrice'])


pd.concat([X_test['Id'],pred],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/DefaultSVR_Results.csv", index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/DefaultSVR_Results.csv')
res.head()

,Id,SalePrice
0,1461,163006.768977
1,1462,163025.197671
2,1463,163037.547001
3,1464,162989.070227
4,1465,162933.041626


In [35]:
#Hyperparameter tuning for SVR
parameters={ 'degree': range(1,20,2),'max_iter' : range(-10,100,20)}
svr_random = RandomizedSearchCV(svr,parameters,n_iter=15)
svr_random.fit(X_train, y)
grid_parm_svr=svr_random.best_params_
print(grid_parm_svr)

#contruct random forest using the best parameters
svr= SVR(**grid_parm_svr)
svr.fit(X_train,y)

#Get the probability of each X_test value is succeed
pred =pd.DataFrame(svr.predict(X_test),columns=["SalePrice"])
#print the probability of the target is success
print(pred.head())

pd.concat([X_test["Id"],pred],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/HyperSVR_Results.csv", index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/HyperSVR_Results.csv')
print(res.head)

{'max_iter': 70, 'degree': 1}
       SalePrice
0  184239.089511
1  184240.467856
2  184239.687152
3  184239.269423
4  184239.137675
<bound method NDFrame.head of         Id      SalePrice
0     1461  184239.089511
1     1462  184240.467856
2     1463  184239.687152
3     1464  184239.269423
4     1465  184239.137675
...    ...            ...
1454  2915  184239.091829
1455  2916  184239.129248
1456  2917  184240.513417
1457  2918  184238.992786
1458  2919  184239.515113

[1459 rows x 2 columns]>


In [36]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, MLP and Decision Tree Regressorr\n")
models = [ GradientBoostingRegressor(), MLPRegressor(), RandomForestRegressor() ]
      
S_Train, S_Test = stacking(models,                   
                           X_train, y, X_test,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
                                        
                           n_folds=4, 
                                                    
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, MLP and Decision Tree Regressorr

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingRegressor]
    fold  0:  [16286.90996001]
    fold  1:  [17032.45833009]
    fold  2:  [15814.53498750]
    fold  3:  [15604.13730205]
    ----
    MEAN:     [16184.51014491] + [548.45603760]
    FULL:     [16184.58042619]

model  1:     [MLPRegressor]
    fold  0:  [29511.35684096]
    fold  1:  [33713.20347753]
    fold  2:  [28710.04269156]
    fold  3:  [29837.74718266]
    ----
    MEAN:     [30443.08754818] + [1932.07542136]
    FULL:     [30442.44806245]

model  2:     [RandomForestRegressor]
    fold  0:  [17570.99739726]
    fold  1:  [18733.30019231]
    fold  2:  [16780.19318681]
    fold  3:  [16827.34887363]
    ----
    MEAN:     [17477.95991250] + [789.732

In [37]:
#Gradient Boosting Regressor================================================================================

abc =GradientBoostingRegressor()
abc.fit(S_Train, y)
pred=pd.DataFrame(abc.predict(S_Test),columns=["SalePrice"])

pd.concat([X_test["Id"],pred],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/GB_ResultsStacking.csv", index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/GB_ResultsStacking.csv')
print(res.head)
search_grid={'n_estimators':range(500,1000,10),'learning_rate':[0.001,0.005]}
#Random Search for hyperparameter tuning
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(S_Train, y)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)

#Construct Gradient Boosting Trees using the best parameters
abc= GradientBoostingRegressor(**grid_parm_abc)
abc.fit(S_Train, y)

#Get the probability of each X_test value is succeed
pred =pd.DataFrame(abc.predict(S_Test),columns=["SalePrice"])
#print the probability of the target is success
print(pred.head())

pd.concat([X_test["Id"],pred],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles/HyperGB_ResultsStacking.csv", index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles/HyperGB_ResultsStacking.csv')
print(res.head)


<bound method NDFrame.head of         Id      SalePrice
0     1461  124089.032242
1     1462  156304.539904
2     1463  182941.399929
3     1464  182941.399929
4     1465  190688.591012
...    ...            ...
1454  2915   79785.368880
1455  2916   90230.941707
1456  2917  147533.575535
1457  2918  127975.616918
1458  2919  230266.492623

[1459 rows x 2 columns]>
{'n_estimators': 670, 'learning_rate': 0.005}
       SalePrice
0  125715.812601
1  160402.443454
2  182746.891715
3  182746.891715
4  189917.777319
<bound method NDFrame.head of         Id      SalePrice
0     1461  125715.812601
1     1462  160402.443454
2     1463  182746.891715
3     1464  182746.891715
4     1465  189917.777319
...    ...            ...
1454  2915   82572.194687
1455  2916   92799.519492
1456  2917  151428.462725
1457  2918  129588.653497
1458  2919  227069.206014

[1459 rows x 2 columns]>


In [38]:
#MLP classifier
#Default mode
mlp = MLPRegressor()
mlp.fit(S_Train,y)

#Get the probability of each X_test value is succeed
pred =pd.DataFrame(mlp.predict(S_Test),columns=['SalePrice'])


pd.concat([X_test['Id'],pred],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/DefaultMLP_ResultsStacking.csv", index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/DefaultMLP_ResultsStacking.csv')
res.head()
#Hyperparameter tuning for MLP
parameters={ 'hidden_layer_sizes': range(80,150,20),'max_iter' : range(10,200,50)}
mlp_random = RandomizedSearchCV(mlp,parameters,n_iter=15)
mlp_random.fit(S_Train, y)
grid_parm_mlp=mlp_random.best_params_
print(grid_parm_mlp)

#contruct random forest using the best parameters
mlp= MLPRegressor(**grid_parm_mlp)
mlp.fit(S_Train,y)

#Get the probability of each X_test value is succeed
pred =pd.DataFrame(mlp.predict(S_Test),columns=["SalePrice"])
#print the probability of the target is success
print(pred.head())

pd.concat([X_test["Id"],pred],axis=1).to_csv("/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles/HyperMLP_ResultsStacking.csv", index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles/HyperMLP_ResultsStacking.csv')
print(res.head)

{'max_iter': 60, 'hidden_layer_sizes': 120}
       SalePrice
0  122603.105844
1  159835.797986
2  187594.544003
3  185803.650589
4  188874.968118
<bound method NDFrame.head of         Id      SalePrice
0     1461  122603.105844
1     1462  159835.797986
2     1463  187594.544003
3     1464  185803.650589
4     1465  188874.968118
...    ...            ...
1454  2915   83144.880072
1455  2916   89745.432659
1456  2917  153036.217465
1457  2918  123873.282494
1458  2919  230380.462061

[1459 rows x 2 columns]>


In [39]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, MLP and Decision Tree Regressor\n")

models = [ GradientBoostingRegressor(), MLPRegressor()]
      
S2_Train, S2_Test = stacking(models,                   
                           S_Train, y, S_Test,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
                                        
                           n_folds=4, 
                                                    
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, MLP and Decision Tree Regressor

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [GradientBoostingRegressor]
    fold  0:  [17890.82829709]
    fold  1:  [18173.48323722]
    fold  2:  [16262.99181169]
    fold  3:  [16330.66455955]
    ----
    MEAN:     [17164.49197639] + [873.72743508]
    FULL:     [17164.99049136]

model  1:     [MLPRegressor]
    fold  0:  [16561.71407787]
    fold  1:  [16947.13580881]
    fold  2:  [15573.02913360]
    fold  3:  [15707.47932667]
    ----
    MEAN:     [16197.33958674] + [575.47583805]
    FULL:     [16197.58967218]



In [40]:

#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL==============================
abc =GradientBoostingRegressor()
#Default mode
abc.fit(S2_Train, y)
abc_predict=abc.predict(S2_Test)


search_grid={'n_estimators':range(500,1000,10),'learning_rate':[0.001,0.005]}
#Grid Search for hyperparameter tuning
abc_random = GridSearchCV(abc,search_grid)
abc_random.fit(S2_Train, y)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)

#Construct Gradient Boosting Trees using the best parametersw
abc= GradientBoostingRegressor(**grid_parm_abc)
abc.fit(S2_Train,y)
abc_predict = abc.predict(S2_Test)





{'learning_rate': 0.005, 'n_estimators': 960}


In [41]:
#run cross-validation on best parameters, get auc score
abc_cv_score = cross_val_score(abc, S2_Train, y, cv=10,scoring="neg_mean_squared_log_error")
print("=== All Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean Score ===")
print("Mean Score - Grid Search: ",abc_cv_score.mean())

=== All Scores ===
[-0.02006997 -0.01596302 -0.01710699 -0.02402687 -0.02791288 -0.01385901
 -0.01941416 -0.0170868  -0.01576073 -0.02136099]


=== Mean Score ===
Mean Score - Grid Search:  -0.019256142091633562


In [42]:
#Get Class Prediction as a data frame with header as Prediction
pred_probability=pd.DataFrame(abc.predict(S2_Test),columns=["SalePrice"])

#back = np.expm1(pred_probability)

In [43]:
#back.head()

In [44]:
#Write into a file with actual prediction and corresponding probability
pd.concat([X_test['Id'],pred_probability],axis=1).to_csv(r'/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/Out_Two_stack_hyp_grd1.csv', index = None)

res=pd.read_csv('/gdrive/Shared drives/The Big Four/CIS 508/TeamAssignment1_ColabFiles1/Out_Two_stack_hyp_grd1.csv')
res.head()

,Id,SalePrice
0,1461,120016.763064
1,1462,165747.283606
2,1463,181725.477382
3,1464,181725.477382
4,1465,190176.533215
